# Models creation tutorial for Debiai example

### Import

In [4]:
# System modules
import importlib
import os
import pathlib

# Tensorflow modules
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

## 1. Models and results functions

#### Here are some functions that will be helpful during the example.

In [5]:
def create_model_from_dir(path, batch_size=32, nb_layers=3):
    """ 
    Create a CNN model from directories of images grouped by labels.
    Return the train and val dataset and the model.
    """
    data_dir = pathlib.Path(path)
    
    # Create a dataset
    img_height = 32
    img_width = 32

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset='training',
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset='validation',
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)
    
    model = create_cnn_model(batch_size=batch_size, nb_layers=nb_layers)
    
    return (train_ds, val_ds, model)
 
def create_cnn_model(batch_size=32, nb_layers=3):
    """ Return a CNN model for 32*32*3 inputs images 
        nb_layers allow to choose number of Conv2D, MaxPooling2D layers
    """
    # Create model
    num_classes = 10
    
    l = []
    
    model = tf.keras.Sequential()
    
    
    model.add(layers.experimental.preprocessing.Rescaling(1./255))
    
    for i in range(nb_layers):
        model.add(layers.Conv2D(32, 3, activation='relu'))
        model.add(layers.MaxPooling2D())
        
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes))

    # Compile model functions
    model.compile(
      optimizer='adam',
      loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

    model.build((batch_size,32,32,3))
    model.summary()
    
    return model

## 2. Create models

#### The first model will also create the train and validation dataset for MNIST

In [6]:
# We get the mnist dataset, validation set and model from this function
(mnist_ds, mnist_val, mnist_model_1) = create_model_from_dir("data/MNIST_reformat/", nb_layers=1)

# We want to try another model with more Conv2D layers
mnist_model_2 = create_cnn_model(nb_layers=3)

# We create datasets and model for MNIST_M
(mnistm_ds, mnistm_val, mnistm_model_1) = create_model_from_dir("data/MNIST_M/train", nb_layers=3)

# Last we can add a model trained on both dataset
# Merge both dataset
full_dataset = mnist_ds.concatenate(mnistm_ds)
full_dataset.shuffle(1)
full_val = mnist_val.concatenate(mnistm_val)

full_model = create_cnn_model(nb_layers=3)

Found 70000 files belonging to 10 classes.
Using 56000 files for training.
Found 70000 files belonging to 10 classes.
Using 14000 files for validation.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (32, 32, 32, 3)           0         
_________________________________________________________________
conv2d (Conv2D)              (32, 30, 30, 32)          896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 15, 15, 32)          0         
_________________________________________________________________
flatten (Flatten)            (32, 7200)                0         
_________________________________________________________________
dense (Dense)                (32, 128)                 921728    
_________________________________________________________________
dense_1 (Dense)              (32, 10

## 3. Train models

In [7]:
# Train the model with mnist (1 layer)
mnist_model_1.fit(mnist_ds, validation_data=mnist_val, epochs=1)

# Train the model with mnist (3 layers)
mnist_model_2.fit(mnist_ds, validation_data=mnist_val, epochs=1)

# Train the model with mnist_m (3 layers)
mnistm_model_1.fit(mnistm_ds, validation_data=mnistm_val, epochs=1)

# Train the model with mnist and mnist_m (3 layers)
full_model.fit(full_dataset,validation_data=full_val,epochs=1)

3428/3428 [==============================] - 974s 284ms/step - loss: 0.2730 - accuracy: 0.9177 - val_loss: 0.1494 - val_accuracy: 0.9540


## 4. Save models

In [8]:
# Save all models in models/ directory
mnist_model_1.save("models/mnist_model_1")
mnist_model_2.save("models/mnist_model_2")
mnistm_model_1.save("models/mnistm_model_1")
full_model.save("models/full_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/mnist_model_1/assets
INFO:tensorflow:Assets written to: models/mnist_model_2/assets
INFO:tensorflow:Assets written to: models/mnistm_model_1/assets
INFO:tensorflow:Assets written to: models/full_model/assets


## CONCLUSION

#### You now have 4 models that will be load and used in the next tutorial which is about how to push data and results into debiai from the package.